# Linkedin Profile Scrapper
*This is **NOT** a real project, it's only used to learning the scrapping concept.*

### Problem Statement:
A school need to know how many and who are the LinkedIn Alumnis.

### Solution:
A web scrapping to find all users in Alumnis section on LinkedIn's School and save it ins a CSV file.

### Libs
- OS - To manipulate local variables
- Time - To wait page loads
- Pandas - To load and manipulate data
- Selenium - To emulate and navegate in a browser
- Scrapy - To scrap data from a website


## Step 1 - Importing libs and load local variables


In [1]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from scrapy import Selector
import random

login_username = os.getenv('LINKEDIN_USER')
password = os.getenv('LINKEDIN_PASSWORD')


## Step 2 - Access url of School's LinkedIn
2.1 Use local variables do login\
2.2 Access the url\
2.3 Scroll a couple of times (important, the number of scroll depends on number of alumnis)


*to do: Get the number of Alumnis to determine the number of scrolls.*

In [5]:
driver = webdriver.Chrome(r"Users\Felipe Mayer\Downloads\chromedriver_win32")

driver.get("https://linkedin.com/uas/login")
  
# waiting for the page to load
time.sleep(5)
  
# entering username
username = driver.find_element(By.XPATH, "//input[@id='username']")

# Enter Your Email Address
username.send_keys(login_username)  
  
# entering password
pword = driver.find_element(By.XPATH, "//input[@id='password']")
  
# Enter Your Password
pword.send_keys(password)        

# Press login button
driver.find_element(By.XPATH, "//button[@class='btn__primary--large from__button--floating']").submit()

# School Linkedin URL
url = "https://www.linkedin.com/school/cursos-pm3/people/"
  
driver.get(url)

driver.maximize_window()

for x in range(1, 200):
    time.sleep(1)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    try:
        driver.find_element(By.XPATH, './/span[text()="Show more results"]/parent::button').click()
    except:
        pass

src = driver.page_source

sel = Selector(text=src)

time.sleep(5)
driver.close()

C:\Users\Felipe Mayer\AppData\Local\Temp\ipykernel_8220\983448309.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"Users\Felipe Mayer\Downloads\chromedriver_win32")


## Step 3 - Get the Alumnis profile urls
3.1 Transform the page source provided by Driver into a Scrapy Selector.\
3.2 Using xpath, get all alumnis url's containers.\
3.3 Iterate over urls and add them to dataframe.\
3.4 Drop URL duplicated

In [6]:
urls = sel.xpath('.//section/div/div/div/a[contains(@href, "/in/")]/@href')

df_alumnis = pd.DataFrame()

for i, url in enumerate(urls):
    df_alumnis.loc[i, 'linkedin'] = url.get()

df_alumnis = df_alumnis.drop_duplicates(keep="last").copy()

df_alumnis

,linkedin
0,https://www.linkedin.com/in/gicosilvaalmeida?m...
1,https://www.linkedin.com/in/falacomcaio?miniPr...
2,https://www.linkedin.com/in/guipaulobarros?min...
3,https://www.linkedin.com/in/felipeperazza?mini...
4,https://www.linkedin.com/in/gabrieltarantelli?...
...,...
993,https://www.linkedin.com/in/joaotorreslima?min...
994,https://www.linkedin.com/in/filipe-oliveira-66...
995,https://www.linkedin.com/in/gabitonunes?miniPr...
996,https://www.linkedin.com/in/emerson-birochi-14...


## Step 4 - Extract the content (as Selector) of each line of dataframe
4.1 Iterate over the dataframe\
4.2 For each iteration, needed to create a new driver (to bypass scrap-blockers)\
4.3 Dismiss the modal if it was created\
4.4 Add a new column in dataframe with the content (selector) of each profile\
4.5 Close Driver to new iteration.

In [ ]:
for i, row in df_alumnis.sample(frac=1/5).iterrows():
        # Creating a webdriver instance
        driver = webdriver.Chrome(r"Users\Felipe Mayer\Downloads\chromedriver_win32")

        # open browser with current user linkedin url as parameter
        driver.get(row['linkedin'])
        time.sleep(random.randint(2, 5))

        try:
            driver.find_element(By.XPATH, '//*[@id="contextual-sign-in"]/div/section/button').click()
        except:
            pass

        src = driver.page_source

        df_alumnis.loc[i, 'selector'] = Selector(text=src)

        driver.close()
        time.sleep(random.randint(2, 5))

df_alumnis

## Step 5 - Extract and organize the content (Selector)

5.1 Drop the na rows, just to avoid errors\
5.2 Itearate over a dataframe\
5.3 Using the xpath get the information.

In [ ]:
df_alumnis = df_alumnis.loc[~df_alumnis['selector'].isna()].reset_index().copy()

for i, row in df_alumnis.iterrows():

    df_alumnis.loc[i, 'status'] = 'Public'
    df_alumnis.loc[i, 'name'] = row['selector'].xpath('//h1/text()').get(default='').replace('\n', ' ').strip()
    df_alumnis.loc[i, 'headline'] = row['selector'].xpath('//h2/text()').get(default='').replace('\n', ' ').strip()
    df_alumnis.loc[i, 'local'] = row['selector'].xpath('//h3/div/text()').get(default='').replace('\n', ' ').strip()
    df_alumnis.loc[i, 'about'] = row['selector'].xpath('//h2[contains(text(),"About")]/following-sibling::div/p/text()').get(default='').replace('\n', ' ').strip()
    df_alumnis.loc[i, 'job_title'] = row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/h3/text()').get(default='').replace('\n', ' ').strip()
    df_alumnis.loc[i, 'current_company'] = row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/h4/a/text()').get(default='').replace('\n', ' ').strip()
    df_alumnis.loc[i, 'job_duration'] =  row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/div/p/span/span/text()').get(default='').replace('\n', ' ').strip()
    df_alumnis.loc[i, 'job_description'] = row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/div/div/div/p/text()').get(default='').replace('\n', ' ').strip()
    df_alumnis.loc[i, 'job_local'] = row['selector'].xpath('//section[contains(@class,"experience")]/div/ul/li[1]/div/div/p[2]/text()').get(default='').replace('\n', ' ').strip()

display(df_alumnis)



## Step 6 - Save as CSV

In [35]:
df_alumnis.to_csv("alumnis.csv", index=False)